In [ ]:
cd /Users/anshupriyasrivastava/Desktop/Udacity/Udacity ML Nanodegree/Fake-Job-Posting-Prediction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
fake_job_postings = pd.read_csv('data/fake_job_postings.csv')

In [ ]:
fake_job_postings.info()

In [ ]:
fake_job_postings.shape

In [ ]:
fake_job_postings.isnull().sum()

In [ ]:
corr = fake_job_postings.corr()
sns.heatmap(corr)
plt.show()

In [ ]:
sns.countplot(x='fraudulent', data=fake_job_postings);

In [ ]:
def sns_countplot(feature):
    sns.countplot(x=feature, data=fake_job_postings, hue="fraudulent",
              order=fake_job_postings[feature].value_counts().iloc[:10].index)
    plt.xticks(rotation=90)
    title = feature + ' fake job count'
    plt.title('Location Fake Jobs')
    plt.show()

In [ ]:
sns_countplot('location')

In [ ]:
sns_countplot('employment_type')

In [ ]:
sns_countplot('required_experience')

In [ ]:
sns_countplot('required_education')

In [ ]:
location_ratio = round(fake_job_postings[fake_job_postings.fraudulent == 1].groupby('location').location.count()/fake_job_postings[fake_job_postings.fraudulent == 0].groupby('location').location.count(), 2)
location_ratio = pd.DataFrame({'location':location_ratio.index, 'ratio':location_ratio.values})

In [ ]:
fake_job_postings = fake_job_postings.merge(location_ratio)

In [ ]:
fake_job_postings.ratio.fillna(0, inplace=True)

In [ ]:
location_ratio_dict = {}

for key, value in zip(fake_job_postings.location, fake_job_postings.ratio):
    location_ratio_dict[key] = value

In [ ]:
location_ratio_df = pd.DataFrame()
location_ratio_df_ratio = []
location_ratio_df_country = []
location_ratio_df_state = []
location_ratio_df_city = []
for key, value in zip(location_ratio_dict.keys(), location_ratio_dict.values()):
    keys = key.split(',')
    print(keys, value)
    # for v in keys:
    #     clean_key = v.strip()
    #     location_ratio_df_country.append(clean_key[0])
    #     location_ratio_df_state.append(clean_key[1])
    #     location_ratio_df_city.append(clean_key[2])
    # location_ratio_df_ratio.append(value)

In [ ]:
location_ratio = location_ratio[location_ratio.ratio >= 1]

In [ ]:
sns.barplot(data=location_ratio.sort_values(by='ratio'), x='location', y='ratio')
plt.xticks(rotation=90)
plt.title('Fake to Real Job Ratio')
plt.show()

In [ ]:
fake_job_postings.columns

In [ ]:
y_axis = fake_job_postings[fake_job_postings.function.isna()][['fraudulent', 'function']]
y_axis = y_axis.fraudulent.value_counts()
y_axis.plot(kind='bar')
plt.xticks(rotation=0)
plt.show()

In [ ]:
def missing_count(feature, title='None'):
    y_axis = fake_job_postings[fake_job_postings[feature].isna()][['fraudulent', feature]]
    y_axis = y_axis.fraudulent.value_counts()
    y_axis.plot(kind='bar')
    plt.ylabel('Count')
    plt.xlabel('Category')
    plt.title(title)
    plt.xticks(rotation=0)
    plt.show()
    return 0

In [ ]:
missing_count('function', 'Function Count')

In [ ]:
missing_count('company_profile', 'Company Profile')

In [ ]:
missing_count('required_education', 'required_education')

In [ ]:
missing_count('industry', 'Industry')

In [ ]:
fake_job_postings[(fake_job_postings.telecommuting == 0) & (fake_job_postings.fraudulent == 1)][['telecommuting', 'fraudulent']]

In [ ]:
fake_job_postings.columns

In [ ]:
telecommuting_list = []
has_company_logo_list = []

for idx, tel, logo in zip(range(len(fake_job_postings)), fake_job_postings.telecommuting, fake_job_postings.has_company_logo):
    if fake_job_postings.fraudulent[idx] == 1:
        telecommuting_list.append(tel)
        has_company_logo_list.append(logo)
    else:
        pass

telecommuting_logo_df = pd.DataFrame({'telecommuting':telecommuting_list, 'has_company_logo':has_company_logo_list})

In [ ]:
fake_count = 0

for fraud, tel, logo in zip(fake_job_postings.fraudulent, fake_job_postings.telecommuting, fake_job_postings.has_company_logo):
    if (tel == 0 and logo == 0):
        if (fraud == 1):
            fake_count +=1
        else:
            pass
    else:
        pass


print(fake_count)

In [ ]:
len(fake_job_postings[fake_job_postings.fraudulent == 1])

In [ ]:
fake_count/len(fake_job_postings[fake_job_postings.fraudulent == 1]) * 100

In [ ]:
fake_job_postings.fillna(" ",inplace = True)

fake_job_postings['text'] =  fake_job_postings['title'] + ' ' + fake_job_postings['location'] + ' ' + fake_job_postings['company_profile'] + ' ' + \
        fake_job_postings['description'] + ' ' + fake_job_postings['requirements'] + ' ' + fake_job_postings['benefits'] + ' ' + \
        fake_job_postings['required_experience'] + ' ' + fake_job_postings['required_education'] + ' ' + fake_job_postings['industry'] + ' ' + fake_job_postings['function']


fake_job_postings.drop(['job_id', 'department', 'salary_range', 'title','location','department','company_profile','description','requirements','benefits','employment_type','required_experience','required_education','industry','function'], axis = 1, inplace = True)

In [ ]:
fake_job_postings['character_count'] = fake_job_postings.text.apply(len)

In [ ]:
fake_job_postings[fake_job_postings.fraudulent==0].character_count.plot(bins=35, kind='hist', color='blue', 
                                       label='Real', alpha=0.8)
fake_job_postings[fake_job_postings.fraudulent==1].character_count.plot(kind='hist', color='red', 
                                       label='Fake', alpha=0.8)
plt.legend()
plt.xlabel("Character Count");

## Text Data Tokenization

In [ ]:
import time
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import RegexpTokenizer

In [ ]:
from nltk.tokenize import word_tokenize
import re

In [ ]:
#pattern_url = "#URL\_d"

fake_job_postings_fraud_only = fake_job_postings[fake_job_postings.fraudulent == 1]
fake_job_postings_not_fraud = fake_job_postings[fake_job_postings.fraudulent == 0]

In [ ]:
tokenized_lines_fraud = [regexp_tokenize(t, "\w+") for t in fake_job_postings_fraud_only.text]
line_num_words_fraud = [len(t_line) for t_line in tokenized_lines_fraud]

In [ ]:
tokenized_lines_not_fraud = [regexp_tokenize(t, "\w+") for t in fake_job_postings_not_fraud.text]
line_num_words_not_fraud = [len(t_line) for t_line in tokenized_lines_not_fraud]
plt.hist(line_num_words_not_fraud, label='Real', color='blue', alpha=0.6)
plt.hist(line_num_words_fraud, label='Fake', color='red')
plt.title('Word Length')
plt.show()

In [ ]:
# Punctuation removal

def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [ ]:
fake_job_postings['text'] = fake_job_postings['text'].apply(lambda x: remove_punctuation(x))

In [ ]:
fake_job_postings.text.head()

In [ ]:
# Toeknization

tokenizer = RegexpTokenizer(r'\w+')
fake_job_postings_tokenized = fake_job_postings
fake_job_postings_tokenized['text'] = fake_job_postings_tokenized['text'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [ ]:
# Removing stopwords

In [ ]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return(words)

In [ ]:
fake_job_postings_tokenized['text'] = fake_job_postings_tokenized['text'].apply(lambda x: remove_stopwords(x))

In [ ]:
# Lemmentizing
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text


fake_job_postings_tokenized['text'] = fake_job_postings_tokenized['text'].apply(lambda x: word_lemmatizer(x))

In [ ]:
# Stemming

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text


fake_job_postings_tokenized['text'] = fake_job_postings_tokenized['text'].apply(lambda x: word_stemmer(x))

In [ ]:
import re

pattern = r'[0-9]'

for text in fake_job_postings_tokenized['text']:
    fake_job_postings_tokenized['text'] = re.sub(pattern, '', text)

In [ ]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(width = 1600 , height = 800 , max_words = 3000).generate(" ".join(fake_job_postings[fake_job_postings.fraudulent == 0].text))
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(width = 1600 , height = 800 , max_words = 3000).generate(" ".join(fake_job_postings[fake_job_postings.fraudulent == 1].text))
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
fake_job_postings.to_csv('data/fake_job_postings_cleaned.csv')